In [329]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys   
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import time
import numpy as np

In [330]:
taglist = ['avocado','avocados','avocadotoast','wherearetheavocados','avocadooil','avocadolove','avocadolover','avocadoplant','avocadosalad','avocadoporn','avocadotree','avocadolovers','avocadoaddict','aguacate']

In [331]:
driver = webdriver.Chrome('/Users/adriaferrer/IronhackRepos/Avocalypse/Tests/chromedriver')

In [332]:
# Define lists to store post information
 
tag_name_list = []
postids_list = []
post_time_list = []
likes_list = []

# Loop over each hashtag to extract information
for tag in taglist:
    print(f'This is #{tag}')
    for scroll in range(100):
        if scroll%10 == 0:
            print(f'This is scroll {scroll}')
        driver.get('https://www.instagram.com/explore/tags/'+str(tag))
        time.sleep(1)
        
        # Extract current hashtag name
        tagname = tag
        
        # Extract all post links from 'explore tags' page
        post_links = []

        if scroll == 0:
            soup = BeautifulSoup(driver.page_source,"lxml")
            recientes = soup.find_all('div', {'class': 'Nnq7C weEfm'})
            recientes = recientes[4:]
            for row in recientes:
                for a in row.find_all('a', href=True):
                    post_links.append(a['href'])            
        else:
            for times in range(scroll):
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(1)
            recientes = driver.find_elements_by_class_name('KC1QD')
            for row in recientes:
                recientes = row.find_elements_by_tag_name('a')
            for a in recientes:
                post_links.append(a.get_attribute('href').strip('https://www.instagram.com'))
            post_links = post_links[9:]

        # Keep link of only 30 most recent posts
        if scroll == 0:
            clean_post_links = [x for x in post_links if x.startswith('/p/')]
        else:
            clean_post_links = ['/p/'+x for x in post_links]
        
        del clean_post_links[30:]

        # Extract the info for each post of the tag
        for postid in range(len(clean_post_links)):

            #Add tagname
            tag_name_list.append(tag)
            driver.get('https://www.instagram.com'+str(clean_post_links[postid]))
            soup = BeautifulSoup(driver.page_source,"lxml")

            #Add post id
            postids_list.append(str(clean_post_links[postid]))

            #Add likes, if any
            likes = soup.find('div', {'class': 'Nm9Fw'})
            if likes == None:
                likes_list.append('0')
            else:
                try:
                    likes_list.append(likes.find('span').text)
                except:
                    likes_list.append('1')

            #Add timestamp
            i = soup.find('time')
            if i == None:
                post_time_list.append(np.nan)
            else:
                post_time_list.append(i['datetime'])
    
driver.quit()

This is #avocado
This is scroll 0
This is scroll 10
This is scroll 20
This is scroll 30
This is scroll 40
This is scroll 50
This is scroll 60
This is scroll 70
This is scroll 80
This is scroll 90
This is #avocados
This is scroll 0
This is scroll 10
This is scroll 20
This is scroll 30
This is scroll 40
This is scroll 50
This is scroll 60
This is scroll 70
This is scroll 80
This is scroll 90
This is #avocadotoast
This is scroll 0
This is scroll 10
This is scroll 20
This is scroll 30
This is scroll 40
This is scroll 50
This is scroll 60
This is scroll 70
This is scroll 80
This is scroll 90
This is #wherearetheavocados
This is scroll 0


KeyboardInterrupt: 

In [333]:
len(postids_list)

4770

In [339]:
tag_name_list_safe = tag_name_list

In [340]:
tag_name_list = tag_name_list[:-1]

In [341]:
len(tag_name_list)

4770

In [334]:
len(set(postids_list))

1218

In [335]:
len(post_time_list)

4770

In [336]:
len(likes_list)

4770

In [342]:
# Add hashtag info to dataframe

tag_df  = pd.DataFrame({'tags': tag_name_list, 'postid': postids_list, 'date': post_time_list, 'likes': likes_list})         

In [346]:
# Check the final dataframe
tag_df.head()
 
# CSV output for hashtag analysis
#tag_df.to_csv('hashtag_list.csv')

,tags,postid,date,likes
0,avocado,/p/Bvt2CQcnYB2/,2019-04-01T14:29:46.000Z,0
1,avocado,/p/Bvt2ANMlhM9/,2019-04-01T14:29:29.000Z,2
2,avocado,/p/Bvt18Zfn2Mb/,2019-04-01T14:29:08.000Z,3
3,avocado,/p/Bvt18PHh1Ch/,2019-04-01T14:28:56.000Z,4
4,avocado,/p/Bvt16cyHNCs/,2019-04-01T14:28:42.000Z,22


In [348]:
tag_df['postid'] = tag_df['postid'].str.strip('/p/')

In [350]:
tag_df['date'] = tag_df['date'].astype('datetime64')

In [354]:
tag_df['likes'] = tag_df['likes'].astype('float64')

In [357]:
unique_tags_df = tag_df.drop_duplicates(subset='postid')

In [366]:
unique_tags_df_nona = unique_tags_df.dropna()

In [369]:
unique_tags_df_nona.shape

(947, 4)

In [370]:
from sqlalchemy import create_engine

In [371]:
driver = 'mysql+pymysql:'
user = 'root'
password = '17028854'
ip = '35.195.111.11'
database = 'Avocalypse'

connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
engine = create_engine(connection_string)

In [372]:
unique_tags_df_nona.to_sql('insta_posts_notnull',con=engine)